In [1]:
import pandas as pd
from collections import defaultdict


In [2]:
df = pd.read_excel("frontier_flights_list.xlsx", sheet_name= "single_destination_coded")

In [3]:

flight_graph = defaultdict(list)

for _, row in df.iterrows():
    origin = row["origin"]
    destination = row["destination"]
    flight_graph[origin].append(destination)


In [4]:
def bfs_destinations(graph, start, max_layovers):
    visited = set()
    queue = [(start, 0)]

    while queue:
        current_airport, layovers = queue.pop(0)
        if layovers > max_layovers:
            continue

        for neighbor in graph.get(current_airport, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, layovers + 1))
    
    visited.discard(start)  # Optional
    return visited

bfs_destinations(flight_graph, 'Norfolk, VA', 0)

set()

In [5]:
def shared_destinations(graph, origin_list, max_layovers):
    all_reachable = [bfs_destinations(graph, origin, max_layovers) for origin in origin_list]
    return set.intersection(*all_reachable) if all_reachable else set()


In [6]:
def round_trip_destinations(graph, start, max_outbound, max_return):
    outbound_reachable = bfs_destinations(graph, start, max_outbound)

    reverse_graph = build_reverse_graph(graph)


    round_trip = set()
    for dest in outbound_reachable:
        returnable = bfs_destinations(reverse_graph, dest, max_return)
        if start in returnable:
            round_trip.add(dest)
    
    return round_trip


def build_reverse_graph(graph):
    reversed_graph = defaultdict(list)
    for origin, destinations in graph.items():
        for dest in destinations:
            reversed_graph[dest].append(origin)
    return reversed_graph


In [7]:
friends = ["DEN", "ATL", "PHX"]
max_layovers = 0

shared = shared_destinations(flight_graph, friends, max_layovers)
print("Shared reachable destinations:", shared)


Shared reachable destinations: {'ORD', 'IAH', 'CLE', 'LAX', 'LAS', 'DTW', 'MCO', 'SFO', 'AUS', 'MSP', 'CVG', 'DFW'}


In [8]:
valid = round_trip_destinations(flight_graph, "ORF", 0, 0)
print(valid)  


{'ATL', 'DFW', 'MCO'}
